In [30]:
import numpy as np
import pandas as pd

%config InlineBackend.figure_format = 'retina'

Read csv files "us-states.csv" and 

In [31]:
covid = pd.read_csv("data/us-states.csv")
covid.head()

,date,state,fips,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths
0,2020-09-13,Alabama,1,137646,2350,124823.0,2217.0,12823.0,133.0
1,2020-09-13,Alaska,2,7074,41,7074.0,41.0,0.0,0.0
2,2020-09-13,Arizona,4,208128,5316,NaN,NaN,NaN,NaN
3,2020-09-13,Arkansas,5,69710,969,68542.0,964.0,1168.0,5.0
4,2020-09-13,California,6,760581,14333,759405.0,14333.0,1176.0,0.0


This dataframe has population estimates and a lot of other info. See nst-est2019-alldata.pdf for a descrition of all columns.

In [50]:
population = pd.read_csv("data/population.csv")
population.head()

,STATE,NAME,POPESTIMATE2019
0,0,United States,328239523
1,0,Northeast Region,55982803
2,0,Midwest Region,68329004
3,0,South Region,125580448
4,0,West Region,78347268


In [33]:
geo_code = pd.read_csv("data/state-geocodes-v2015.csv")

In [53]:
geo_code

,Region,Division,fips,Name
0,1,0,0,Northeast Region
1,1,1,0,New England Division
2,1,1,9,Connecticut
3,1,1,23,Maine
4,1,1,25,Massachusetts
...,...,...,...,...
59,4,9,2,Alaska
60,4,9,6,California
61,4,9,15,Hawaii
62,4,9,41,Oregon


### Q1: How many states have had more between 500  and 1000 deaths?

In [35]:
#my answer
len(covid[(covid["deaths"] >= 500) & (covid["deaths"] <= 1000)])

8

In [36]:
#Their answer
((covid.deaths > 500) & (covid.deaths < 1000)).sum()

8

### Q2: what is the rate of cases of covid per 1000 people.

In [37]:
#My answer
1000 * covid["cases"].sum() / population['POPESTIMATE2019'].iloc[0]

19.816857947359374

In [38]:
n_pop = population.loc[0, "POPESTIMATE2019"]
n_pop

328239523

In [39]:
1000*covid["cases"].sum()/328239523

19.816857947359374

In [40]:
covid["cases"].sum()

6504676

### Q3: Compute the median rate of cases of covid per 1000 people per state.

In [41]:
(covid["cases"] / 1000).median() #******RATE means cases/total population*******
df = covid.merge(population, how = "inner", left_on = "fips", right_on = "STATE")
1000 * (df['cases'] / df['POPESTIMATE2019']).median()

17.719143228570545

In [42]:
df = covid.merge(population, how="inner", left_on="fips", right_on="STATE")
df.head()

,date,state,fips,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths,STATE,NAME,POPESTIMATE2019
0,2020-09-13,Alabama,1,137646,2350,124823.0,2217.0,12823.0,133.0,1,Alabama,4903185
1,2020-09-13,Alaska,2,7074,41,7074.0,41.0,0.0,0.0,2,Alaska,731545
2,2020-09-13,Arizona,4,208128,5316,NaN,NaN,NaN,NaN,4,Arizona,7278717
3,2020-09-13,Arkansas,5,69710,969,68542.0,964.0,1168.0,5.0,5,Arkansas,3017804
4,2020-09-13,California,6,760581,14333,759405.0,14333.0,1176.0,0.0,6,California,39512223


In [43]:
df["rate_cases"] = 1000*df["cases"]/df["POPESTIMATE2019"]

In [44]:
df["rate_cases"].median()

17.719143228570545

### Q4: Compute the rates of confimed cases per 1000 by state.
If "confimed_cases" is NA please use "cases". Give the top worst states

In [48]:
data = {
    "state": df["state"], 
    "fips": df["fips"], 
    "rate_confirmed_cases": 1000 * (df['confirmed_cases'].fillna(df['cases']) / df['POPESTIMATE2019'])
}
conf_cases = pd.DataFrame(data)
conf_cases.sort_values(by = 'rate_confirmed_cases', ascending = False).head(5)[["state", "rate_confirmed_cases"]]

,state,rate_confirmed_cases
18,Louisiana,33.594519
2,Arizona,28.594050
24,Mississippi,28.305706
10,Georgia,26.025053
0,Alabama,25.457534


In [24]:
df["cc"] = df["confirmed_cases"]
df.cc = df.cc.fillna(value=df.cases)

In [25]:
df["rates_cases"] = 1000*df["cc"]/df["POPESTIMATE2019"]

In [29]:
df2 = df[["state","rates_cases"]].copy()

In [30]:
df2 = df2.sort_values(by=["rates_cases"], ascending=False)

In [31]:
df2.iloc[:5]

,state,rates_cases
18,Louisiana,33.594519
2,Arizona,28.594050
24,Mississippi,28.305706
10,Georgia,26.025053
0,Alabama,25.457534


### Q5:  Compute rates of cases per 1000 per region. 
Use "state-geocodes-v2015.csv" to get codes per region.
What is the region with the highest rate?

In [77]:
reg = geo_code.merge(df, how = "inner", left_on = "fips", right_on = "fips")
new_reg = reg.groupby("Region").sum()[["cases", "POPESTIMATE2019"]]
reg2 = new_reg.reset_index().merge(geo_code[(geo_code['Division'] == 0) & (geo_code['fips'] == 0)], how = "inner", left_on = "Region", right_on = "Region")
reg2 = reg2[["Region", "cases", "POPESTIMATE2019", "Name"]]
reg2['rates'] = 1000 * reg2['cases'] / reg2['POPESTIMATE2019']
reg2.sort_values(by = 'rates', ascending = False).iloc[0]["Name"]

'South Region'

In [32]:
df3 = df[["state","cases", "POPESTIMATE2019", "fips", ]].copy()

In [33]:
geo = pd.read_csv("state-geocodes-v2015.csv")

In [34]:
df4 = df3.merge(geo, how="inner", left_on="fips", right_on="fips")
df4.head()

,state,cases,POPESTIMATE2019,fips,Region,Division,Name
0,Alabama,137646,4903185,1,3,6,Alabama
1,Alaska,7074,731545,2,4,9,Alaska
2,Arizona,208128,7278717,4,4,8,Arizona
3,Arkansas,69710,3017804,5,3,7,Arkansas
4,California,760581,39512223,6,4,9,California


In [35]:
agg = df4.groupby("Region").sum()

In [36]:
agg

,cases,POPESTIMATE2019,fips,Division
Region,,,,
1,1011042,55982803,296,12
2,1102292,68329004,365,43
3,2984184,125580448,469,97
4,1365963,78347268,347,109


In [37]:
agg["rates"] = 1000*agg["cases"]/agg["POPESTIMATE2019"]

In [38]:
agg

,cases,POPESTIMATE2019,fips,Division,rates
Region,,,,,
1,1011042,55982803,296,12,18.059867
2,1102292,68329004,365,43,16.132125
3,2984184,125580448,469,97,23.763126
4,1365963,78347268,347,109,17.434724
